In [1]:
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import csr_matrix

from models.group_recommender_mf import GroupRecommenderMF
from models.explicit_mf_with_bias import SGDExplicitBiasMF

In [2]:
anime_df = pd.read_csv("../../data/anime.csv")

In [3]:
group_mf = GroupRecommenderMF(
    full_model_file_path = "../../data/model_sgd_mf_v4_50__1666837325.pkl",
    item_encoder_file_path="../../data/anime_encoder.csv")

In [4]:
group_mf.item_vecs.shape

(5569, 128)

In [5]:
group_mf.item_encoder_df

,original_id,encoded_id
0,1,0
1,6,1
2,7,2
3,8,3
4,15,4
...,...,...
5564,45615,5564
5565,45616,5565
5566,46000,5566
5567,47398,5567


In [6]:
group_rating_df = pd.read_csv("../../data/group_rating_real.csv")
group_rating_df.columns = ["user_name", "item_id", "rating"]
group_rating_df

,user_name,item_id,rating
0,Kien,20,9
1,Kien,1735,8
2,Kien,918,10
3,Kien,21,6
4,Kien,30276,10
5,Kien,25777,10
6,Kien,40028,9
7,Ajay Friend,30276,10
8,Ajay Friend,20,9
9,Ajay Friend,1735,8


In [7]:
# Try different regularization parameters
for reg in [1e-2, 1e-1, 1, 1e1, 1e2]:
    rec_type = "virtual_user"
    agg_method = "mean"
    recommendations = group_mf.recommend_group(group_rating_df, reg=reg, 
        rec_type=rec_type, agg_method=agg_method)
    print("reg = {}, rec_type = {}, agg_method = {}".format(reg, rec_type, agg_method))
    print(recommendations)


reg = 0.01, rec_type = virtual_user, agg_method = mean
   recommendation_score       Kien  Ajay Friend  Lily‘s friend    Tanmay  \
0             20.460308  19.602449     7.354350       7.425789  7.006823   
1             19.812627  15.675646     7.997404       7.553798  7.833024   
2             18.890997  16.336947     8.290082       8.129752  7.947976   
3             18.569657  17.894413     7.466569       7.823185  7.135932   
4             18.528714  14.671626     7.664979       7.671208  7.177290   
5             18.012909  17.515401     8.616562       8.365803  7.971952   
6             17.627740  11.115469     8.077318       7.571617  7.433633   
7             17.419138  16.792185     8.778013       7.467039  8.365394   
8             17.353638  15.918980     8.629981       7.498418  8.536153   
9             17.326561  18.099235     8.788678       8.319542  8.270039   

   Tanmay's friend     Chase  item_id  
0         9.296360  6.515234    16774  
1         6.561991  8.616257

In [8]:
# Try different recommendation types (rec_type)
for rec_type in ["virtual_user", "combine_recommender"]:
    reg = 1
    agg_method = "mean"
    recommendations = group_mf.recommend_group(group_rating_df, reg=reg, 
        rec_type=rec_type, agg_method=agg_method)
    print("reg = {}, rec_type = {}, agg_method = {}".format(reg, rec_type, agg_method))
    print(recommendations)


reg = 1, rec_type = virtual_user, agg_method = mean
   recommendation_score      Kien  Ajay Friend  Lily‘s friend    Tanmay  \
0             10.854972  7.982945     8.233891       7.620590  8.211658   
1             10.745583  8.054483     8.611311       8.517568  8.127477   
2             10.531300  8.472874     8.476894       7.372219  8.182655   
3             10.276055  6.311831     6.383707       6.063359  6.653133   
4             10.201239  8.673754     8.611553       7.518127  8.520127   
5             10.021067  7.226166     7.672293       7.666286  7.190326   
6              9.965378  7.608402     7.754538       7.081450  7.997534   
7              9.919194  8.369005     8.530808       8.205743  8.147707   
8              9.819855  6.948370     6.870567       7.435586  7.684697   
9              9.739498  8.488874     8.699859       8.462360  8.389898   

   Tanmay's friend     Chase  item_id  
0         9.236191  8.147362     7724  
1         8.331187  8.851320       30  
2 

In [10]:
# Try different agg_method (rec_type)
for agg_method in ["min", "mean", "max"]:
    reg = 1
    rec_type = "combine_recommender"
    recommendations = group_mf.recommend_group(group_rating_df, reg=reg, 
        rec_type=rec_type, agg_method=agg_method, k=20)
    print("reg = {}, rec_type = {}, agg_method = {}".format(reg, rec_type, agg_method))
    print(recommendations)


reg = 1, rec_type = combine_recommender, agg_method = min
    recommendation_score      Kien  Ajay Friend  Lily‘s friend    Tanmay  \
0               8.531909  9.037251     8.996379       8.818661  9.031197   
1               8.389898  8.488874     8.699859       8.462360  8.389898   
2               8.368869  8.408205     8.694580       8.368869  8.691821   
3               8.209148  9.010242     9.153101       8.729383  8.600528   
4               8.204586  8.204586     8.262421       8.729112  8.325559   
5               8.199704  8.409818     8.536208       8.797864  8.561612   
6               8.170332  8.324277     8.600052       8.621617  8.328987   
7               8.133889  8.814287     8.787680       8.335666  8.277312   
8               8.099070  8.739240     9.049786       8.349207  8.585627   
9               8.085188  8.099900     8.476505       8.311946  8.086816   
10              8.084073  8.919159     9.017130       8.084073  8.871408   
11              8.082335  8.12

In [11]:
recommendations.merge(anime_df, left_on="item_id", right_on="MAL_ID", how="inner")[
    ["item_id", "Name", "English name", "recommendation_score"]
]

,item_id,Name,English name,recommendation_score
0,38524,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,10.058645
1,28825,Himouto! Umaru-chan,Himouto! Umaru-chan,9.965230
2,9969,Gintama',Gintama Season 2,9.926534
3,32995,Yuri!!! on Ice,Yuri!!! On ICE,9.886779
4,15417,Gintama': Enchousen,Gintama:Enchousen,9.802084
5,35760,Shingeki no Kyojin Season 3,Attack on Titan Season 3,9.745201
6,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist:Brotherhood,9.733310
7,11061,Hunter x Hunter (2011),Hunter x Hunter,9.641880
8,34096,Gintama.,Gintama Season 5,9.637358
9,6500,Seikon no Qwaser,The Qwaser of Stigmata,9.574590
